In [1]:
import urllib
import random, string
import jsonlines

In [2]:
from etk.etk import ETK
from etk.extractors.html_content_extractor import HTMLContentExtractor, Strategy
from etk.extractors.html_metadata_extractor import HTMLMetadataExtractor
from etk.etk_module import ETKModule
from etk.extractors.date_extractor import DateExtractor
from etk.extractors.glossary_extractor import GlossaryExtractor
from etk.extractors.table_extractor import TableExtractor
import json


In [3]:
def get_random_doc_id():
    x = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(16))
    return x

In [4]:
url_list = ["https://www.bloomberg.com/markets/stocks", "https://www.bloomberg.com/markets", "https://www.bloomberg.com/asia","https://money.cnn.com/data/markets/","https://www.kiplinger.com/","https://www.thisismoney.co.uk/money/index.html","https://www.marketwatch.com","https://www.forbes.com/money/","https://www.nytimes.com/section/business/dealbook","https://www.mymoney.gov/Pages/default.aspx"]

In [5]:
for url in url_list:
    print(url + '\n')

https://www.bloomberg.com/markets/stocks

https://www.bloomberg.com/markets

https://www.bloomberg.com/asia

https://money.cnn.com/data/markets/

https://www.kiplinger.com/

https://www.thisismoney.co.uk/money/index.html

https://www.marketwatch.com

https://www.forbes.com/money/

https://www.nytimes.com/section/business/dealbook

https://www.mymoney.gov/Pages/default.aspx



In [6]:
def get_raw_text(url):
    f = urllib.request.urlopen(url)
    s = f.read().decode("utf-8")
    return s

In [7]:
with jsonlines.open("bloomberg_market.jl",'w') as writer:
    for url in ["https://www.bloomberg.com/markets"]: 
        temp = dict()
        print(url)
        temp['url'] = url
        temp["doc_id"] = get_random_doc_id()
        temp['raw_content'] = get_raw_text(url)
        
        writer.write(temp)

https://www.bloomberg.com/markets


## Reading the HTML file

We read the url, get the body of response. We also create a `cdr`. It contains `raw_content` and `url` field.

# First URL

In [8]:
url = "https://www.bloomberg.com/markets"

raw_content = get_raw_text(url)

cdr = {
    'raw_content': raw_content,
    'url': url
}

print('The first 600 chars of the html page:\n')
print(raw_content[:600])

The first 600 chars of the html page:

<!doctype html>
<html>
<head>
    <title>Bloomberg - Are you a robot?</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" type="text/css" href="https://assets.bwbx.io/font-service/css/BWHaasGrotesk-55Roman-Web,BWHaasGrotesk-75Bold-Web,BW%20Haas%20Text%20Mono%20A-55%20Roman/font-face.css">
    <style rel="stylesheet" type="text/css">
        html, body, div, span, applet, object, iframe,
        h1, h2, h3, h4, h5, h6, p, blockquote, pre,
        a, abbr, acronym, address, big, cite, code,
        del, dfn, em, img, ins, kbd, q, s, samp,
 


In [9]:
my_table_extractor = TableExtractor()
tables_in_page = my_table_extractor.extract(raw_content)[:14]
print('Number of tables in this page:', len(tables_in_page), '\n')
print('The first table in the page shows below: \n')

if len(tables_in_page) >0:
    print(json.dumps(tables_in_page[0].value, indent=2))
else:
    print("No tables are present in HTML page")

Number of tables in this page: 0 

The first table in the page shows below: 

No tables are present in HTML page


# Second URL

In [10]:
url = "https://www.bloomberg.com/markets/stocks"

raw_content = get_raw_text(url)

cdr = {
    'raw_content': raw_content,
    'url': url
}

print('The first 600 chars of the html page:\n')
print(raw_content[:600])

The first 600 chars of the html page:

<!doctype html>
<html>
<head>
    <title>Bloomberg - Are you a robot?</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" type="text/css" href="https://assets.bwbx.io/font-service/css/BWHaasGrotesk-55Roman-Web,BWHaasGrotesk-75Bold-Web,BW%20Haas%20Text%20Mono%20A-55%20Roman/font-face.css">
    <style rel="stylesheet" type="text/css">
        html, body, div, span, applet, object, iframe,
        h1, h2, h3, h4, h5, h6, p, blockquote, pre,
        a, abbr, acronym, address, big, cite, code,
        del, dfn, em, img, ins, kbd, q, s, samp,
 


In [11]:
my_table_extractor = TableExtractor()
tables_in_page = my_table_extractor.extract(raw_content)[:14]
print('Number of tables in this page:', len(tables_in_page), '\n')
print('The first table in the page shows below: \n')

if len(tables_in_page) >0:
    print(json.dumps(tables_in_page[0].value, indent=2))
else:
    print("No tables are present in HTML page")

Number of tables in this page: 0 

The first table in the page shows below: 

No tables are present in HTML page


## Inferlink Extractor

This class extracts segments from an HTML page using rules created by the Inferlink web wrapper.

In [12]:
import os, sys, json, codecs

from etk.etk import ETK
from etk.extractors.inferlink_extractor import InferlinkExtractor, InferlinkRuleSet
from etk.etk_module import ETKModule


class InferlinkETKModule(ETKModule):
    """
    Abstract class for extraction module
    """
    def __init__(self, etk):
        ETKModule.__init__(self, etk)
        self.inferlink_extractor = InferlinkExtractor(
            InferlinkRuleSet(InferlinkRuleSet.load_rules_file('sample_inferlink_rules.json')))

    def process_document(self, doc):
        """
        Add your code for processing the document
        """

        raw = doc.select_segments("$.raw_content")[0]
        extractions = doc.extract(self.inferlink_extractor, raw)
        doc.store(extractions, "inferlink_extraction")
        return list()


if __name__ == "__main__":
    sample_html = json.load(codecs.open('sample_html.json', 'r')) # read sample file from disk

    etk = ETK(modules=InferlinkETKModule)
    doc = etk.create_document(sample_html, mime_type="text/html", url="http://ex.com/123")

    docs = etk.process_ems(doc)

    print(json.dumps(docs[0].value, indent=2))

INFO:ETK: processing with <class '__main__.InferlinkETKModule'>. Process doc_id: 1A4A5FF5BD066309C72C8EEE6F7BCCCFD21B83245AFCDADDF014455BCF990A21 url: http://ex.com/123


{
  "crawl_data": {
    "status": 200
  },
  "timestamp": "2017-01-02T18:02:45",
  "content_type": "application/xhtml+xml; charset=ISO-8859-1",
  "url": "http://www.eroticmugshots.com/losangeles-escorts/323-452-2013/?pid=89701497",
  "raw_content": "\n<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Strict//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd\">\n<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\" xml:lang=\"en\">\n<head>\n<title>323-452-2013 ESCORT ALERT! - Luna The Hot Playmate (323) 452-2013 - 23</title>\n<meta name=\"netinsert\" content=\"0.0.1.13.13.1\">\n<meta name=\"description\" content=\"323-452-2013 Escort listings. View all of her listings at once. On this page you will see a history for 323-452-2013 and all of the escort ads placed with this phone number. Specifically in losangeles: 323-452-2013 ESCORT ALERT! - Luna The Hot Playmate (323) 452-2013 - 23\"/>\n<meta name=\"keywords\" content=\"323-452-2013, escort listing, escort ad, escorts in Los angel

In [13]:
print(json.dumps(json.load(codecs.open('sample_html.json', 'r')), indent=2))

{
  "crawl_data": {
    "status": 200
  },
  "timestamp": "2017-01-02T18:02:45",
  "content_type": "application/xhtml+xml; charset=ISO-8859-1",
  "url": "http://www.eroticmugshots.com/losangeles-escorts/323-452-2013/?pid=89701497",
  "raw_content": "\n<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Strict//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd\">\n<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\" xml:lang=\"en\">\n<head>\n<title>323-452-2013 ESCORT ALERT! - Luna The Hot Playmate (323) 452-2013 - 23</title>\n<meta name=\"netinsert\" content=\"0.0.1.13.13.1\">\n<meta name=\"description\" content=\"323-452-2013 Escort listings. View all of her listings at once. On this page you will see a history for 323-452-2013 and all of the escort ads placed with this phone number. Specifically in losangeles: 323-452-2013 ESCORT ALERT! - Luna The Hot Playmate (323) 452-2013 - 23\"/>\n<meta name=\"keywords\" content=\"323-452-2013, escort listing, escort ad, escorts in Los angel

In [14]:
print(json.dumps(json.load(codecs.open('sample_inferlink_rules.json', 'r')), indent=2))

{
  "metadata": {
    "task_name": "project1-_-example.org-20180607_211730",
    "tld": "example.org"
  },
  "rules": [
    {
      "rule_type": "ItemRule",
      "removehtml": true,
      "strip_end_regex": "\\</h2\\>\\s+\\</",
      "begin_regex": ";\\s+margin:none;\\s+display.*?:inline;\"\\>",
      "end_regex": "\\</h2\\>\\s+\\</",
      "include_end_regex": true,
      "name": "inferlink_phone"
    },
    {
      "rule_type": "ItemRule",
      "removehtml": true,
      "strip_end_regex": "\\</span\\>\\s+\\<br",
      "begin_regex": "\"\\>Escort's\\s+Location.*?span\\s+class=\"mugshotData\".*?\\>",
      "end_regex": "\\</span\\>\\s+\\<br",
      "include_end_regex": true,
      "name": "inferlink_location"
    },
    {
      "rule_type": "ItemRule",
      "removehtml": true,
      "strip_end_regex": "\\</span\\>\\s+\\<br",
      "begin_regex": "\"mugshotDataCat\"\\>Escort's\\s+Age.*?span\\s+class=\"mugshotData\".*?\\>",
      "end_regex": "\\</span\\>\\s+\\<br",
      "include_end

In [15]:
import os, sys, json, codecs
from etk.etk import ETK
from etk.extractors.inferlink_extractor import InferlinkExtractor, InferlinkRuleSet
from etk.etk_module import ETKModule


class InferlinkETKModule(ETKModule):
    """
    Abstract class for extraction module
    """
    def __init__(self, etk):
        ETKModule.__init__(self, etk)
        self.inferlink_extractor = InferlinkExtractor(
            InferlinkRuleSet(InferlinkRuleSet.load_rules_file('sample_inferlink_rules.json')))

    def process_document(self, doc):
        """
        Add your code for processing the document
        """

        raw = doc.select_segments("$.raw_content")[0]
        extractions = doc.extract(self.inferlink_extractor, raw)
        print("444444")
        print(extractions)
        print("444444")
        
        doc.store(extractions, "inferlink_extraction")
        return list()


if __name__ == "__main__":
    sample_html = cdr # read sample file from disk

    etk = ETK(modules=InferlinkETKModule)
    doc = etk.create_document(sample_html, mime_type="text/html", url="http://ex.com/123")

    docs = etk.process_ems(doc)

    print(json.dumps(docs[0].value, indent=2))

INFO:ETK: processing with <class '__main__.InferlinkETKModule'>. Process doc_id: None url: http://ex.com/123


444444
[]
444444
{
  "raw_content": "<!doctype html>\n<html>\n<head>\n    <title>Bloomberg - Are you a robot?</title>\n    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1\">\n    <link rel=\"stylesheet\" type=\"text/css\" href=\"https://assets.bwbx.io/font-service/css/BWHaasGrotesk-55Roman-Web,BWHaasGrotesk-75Bold-Web,BW%20Haas%20Text%20Mono%20A-55%20Roman/font-face.css\">\n    <style rel=\"stylesheet\" type=\"text/css\">\n        html, body, div, span, applet, object, iframe,\n        h1, h2, h3, h4, h5, h6, p, blockquote, pre,\n        a, abbr, acronym, address, big, cite, code,\n        del, dfn, em, img, ins, kbd, q, s, samp,\n        small, strike, strong, sub, sup, tt, var,\n        b, u, i, center,\n        dl, dt, dd, ol, ul, li,\n        fieldset, form, label, legend,\n        table, caption, tbody, tfoot, thead, tr, th, td,\n        article, aside, canvas, details, embed,\n        figure, figcaption, footer, header, hgroup,\n        menu, nav, output, ru

## Regex ETK Module

In [16]:
import json, os, sys

from etk.etk import ETK
from etk.extractors.bitcoin_address_extractor import BitcoinAddressExtractor
from etk.extractors.cryptographic_hash_extractor import CryptographicHashExtractor
from etk.extractors.cve_extractor import CVEExtractor
from etk.extractors.hostname_extractor import HostnameExtractor
from etk.extractors.ip_address_extractor import IPAddressExtractor
from etk.extractors.url_extractor import URLExtractor
from etk.etk_module import ETKModule


class RegexETKModule(ETKModule):
    """
    Abstract class for extraction module
    """

    def __init__(self, etk):
        ETKModule.__init__(self, etk)
        bae = BitcoinAddressExtractor()
        ce = CVEExtractor()
        che = CryptographicHashExtractor()
        he = HostnameExtractor()
        iae = IPAddressExtractor()
        ue = URLExtractor(True)
        self.e_list = [bae, ce, che, he, iae, ue]

    def process_document(self, doc):
        """
        Add your code for processing the document
        """

        segment = doc.select_segments("target_text")[0]

        for e in self.e_list:
            res = doc.extract(e, segment)
            doc.store(res, e.name)
        return list()


if __name__ == "__main__":
    sample_input = {
        "target_text": "This is for test multiple extractors using regex expression. "
                       "A module can be used for bitcoin address extraction is "
                       "https://github.com/nederhoed/python-bitcoinaddress, but import is always failed. "
                       "The auther said 'If you use this module, support me with bitcoins! "
                       "Any amount is appreciated. 1qYsJbtEWAeXMsbgxUgGsJsAp3VArsBRd'. "
                       "SHA1(The quick brown fox jumps over the lazy dog)\ngives hexadecimal: "
                       "2fd4e1c67a2d28fced849ee1bb76e7391b93eb12, SHA1('aaa')=7e240de74fb1ed08fa08d38063f6a6a91462a815,"
                       "and a MD5 sample can be e4d909c290d0fb1ca068ffaddf22cbd0. Some CVE can be CVE-1993-1344,  "
                       "CVE-2006-1232 and CVE-1993-1344. We search through "
                       "https://www.google.com and go shopping with amazon.com, sites brun1989.itldc-customer.net."
                       "my ip address is 192.168.0.1 , 193.14.1.1 and 193.14.1.1 not 193.14"
                       "https://foo_bar.example.com/   %%%###http://isi.edu/abx  dig.isi.edu/index.html"
    }

    etk = ETK(modules=RegexETKModule)
    doc = etk.create_document(sample_input)

    docs = etk.process_ems(doc)

    print(json.dumps(docs[0].value, indent=2))


INFO:ETK: processing with <class '__main__.RegexETKModule'>. Process doc_id: None url: http://ex.com/123


{
  "target_text": "This is for test multiple extractors using regex expression. A module can be used for bitcoin address extraction is https://github.com/nederhoed/python-bitcoinaddress, but import is always failed. The auther said 'If you use this module, support me with bitcoins! Any amount is appreciated. 1qYsJbtEWAeXMsbgxUgGsJsAp3VArsBRd'. SHA1(The quick brown fox jumps over the lazy dog)\ngives hexadecimal: 2fd4e1c67a2d28fced849ee1bb76e7391b93eb12, SHA1('aaa')=7e240de74fb1ed08fa08d38063f6a6a91462a815,and a MD5 sample can be e4d909c290d0fb1ca068ffaddf22cbd0. Some CVE can be CVE-1993-1344,  CVE-2006-1232 and CVE-1993-1344. We search through https://www.google.com and go shopping with amazon.com, sites brun1989.itldc-customer.net.my ip address is 192.168.0.1 , 193.14.1.1 and 193.14.1.1 not 193.14https://foo_bar.example.com/   %%%###http://isi.edu/abx  dig.isi.edu/index.html",
  "provenances": [
    {
      "@id": 0,
      "@type": "extraction_provenance_record",
      "method": "bit

## Basic KG Building

In [17]:
import os, sys
from etk.etk import ETK
from etk.knowledge_graph import KGSchema
from etk.extractors.glossary_extractor import GlossaryExtractor
from etk.etk_module import ETKModule
import json


class ExampleETKModule(ETKModule):
    """
    Abstract class for extraction module
    """
    def __init__(self, etk):
        ETKModule.__init__(self, etk)
        self.name_extractor = GlossaryExtractor(self.etk.load_glossary("./names.txt"), "name_extractor",
                                                self.etk.default_tokenizer,
                                                case_sensitive=False, ngrams=1)

    def process_document(self, doc):
        """
        Add your code for processing the document
        """

        descriptions = doc.select_segments("projects[*].description")
        projects = doc.select_segments("projects[*]")

        for d, p in zip(descriptions, projects):
            names = doc.extract(self.name_extractor, d)
            p.store(names, "members")

        doc.kg.add_value("developer", json_path="projects[*].members[*]")
        return list()

if __name__ == "__main__":

    sample_input = {
        "projects": [
            {
                "name": "etk",
                "description": "version 2 of etk, implemented by Runqi12 Shao, Dongyu Li, Sylvia lin, Amandeep and others."
            },
            {
                "name": "rltk",
                "description": "record linkage toolkit, implemented by Pedro, Mayank, Yixiang and several students."
            }
        ]
    }

    kg_schema = KGSchema(json.load(open("master_config.json", "r")))
    etk = ETK(kg_schema=kg_schema, modules=ExampleETKModule)
    doc = etk.create_document(sample_input)

    docs = etk.process_ems(doc)

    print(json.dumps(docs[0].kg.value, indent=2))


INFO:ETK: processing with <class '__main__.ExampleETKModule'>. Process doc_id: None url: http://ex.com/123


{
  "developer": [
    {
      "value": "Dongyu",
      "key": "dongyu"
    },
    {
      "value": "Sylvia",
      "key": "sylvia"
    },
    {
      "value": "Amandeep",
      "key": "amandeep"
    },
    {
      "value": "Pedro",
      "key": "pedro"
    },
    {
      "value": "Mayank",
      "key": "mayank"
    },
    {
      "value": "Yixiang",
      "key": "yixiang"
    }
  ]
}


## HTMLContentExtractor and HTMLMetadataExtractor

In [18]:
import os, sys, json, codecs
from etk.etk import ETK
from etk.extractors.html_content_extractor import HTMLContentExtractor, Strategy
from etk.extractors.html_metadata_extractor import HTMLMetadataExtractor
from etk.etk_module import ETKModule


class HtmlBasicETKModule(ETKModule):
    """
    Abstract class for extraction module
    """
    def __init__(self, etk):
        ETKModule.__init__(self, etk)
        self.metadata_extractor = HTMLMetadataExtractor()
        self.content_extractor = HTMLContentExtractor()

    def process_document(self, doc):
        """
        Add your code for processing the document
        """

        raw = doc.select_segments("$.raw_content")[0]

        doc.store(doc.extract(self.content_extractor, raw, strategy=Strategy.ALL_TEXT), "etk2_text")
        doc.store(doc.extract(self.content_extractor, raw, strategy=Strategy.MAIN_CONTENT_STRICT),
                              "etk2_content_strict")
        doc.store(doc.extract(self.content_extractor, raw, strategy=Strategy.MAIN_CONTENT_RELAXED),
                              "etk2_content_relaxed")
        doc.store(doc.extract(self.metadata_extractor,
                              raw,
                              extract_title=True,
                              extract_meta=True,
                              extract_microdata=True,
                              extract_rdfa=True,
                              ), "etk2_metadata")
        return list()


if __name__ == "__main__":

    sample_html = json.load(codecs.open('sample_html.json', 'r')) # read sample file from disk

    etk = ETK(modules=HtmlBasicETKModule)
    doc = etk.create_document(sample_html, mime_type="text/html", url="http://ex.com/123")

    docs= etk.process_ems(doc)

    print(json.dumps(docs[0].value, indent=2))


INFO:ETK: processing with <class '__main__.HtmlBasicETKModule'>. Process doc_id: 1A4A5FF5BD066309C72C8EEE6F7BCCCFD21B83245AFCDADDF014455BCF990A21 url: http://ex.com/123


{
  "crawl_data": {
    "status": 200
  },
  "timestamp": "2017-01-02T18:02:45",
  "content_type": "application/xhtml+xml; charset=ISO-8859-1",
  "url": "http://www.eroticmugshots.com/losangeles-escorts/323-452-2013/?pid=89701497",
  "raw_content": "\n<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Strict//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd\">\n<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\" xml:lang=\"en\">\n<head>\n<title>323-452-2013 ESCORT ALERT! - Luna The Hot Playmate (323) 452-2013 - 23</title>\n<meta name=\"netinsert\" content=\"0.0.1.13.13.1\">\n<meta name=\"description\" content=\"323-452-2013 Escort listings. View all of her listings at once. On this page you will see a history for 323-452-2013 and all of the escort ads placed with this phone number. Specifically in losangeles: 323-452-2013 ESCORT ALERT! - Luna The Hot Playmate (323) 452-2013 - 23\"/>\n<meta name=\"keywords\" content=\"323-452-2013, escort listing, escort ad, escorts in Los angel